In [1]:
# First clone and set up: https://github.com/dwalton76/rubiks-cube-NxNxN-solver.
# Then, put this notebook in the root directory of that repo.

# General libraries
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import json

In [25]:
DATA_DIRECTORY = '../santa-2023/'

In [26]:
# Rubik's Cube solver library from GitHub
from rubikscubennnsolver import RubiksCube, SolveError
import rubikscubennnsolver
from rubikscubennnsolver.RubiksCubeNNNOdd import RubiksCubeNNNOdd
from rubikscubennnsolver.RubiksCubeNNNEven import RubiksCubeNNNEven

In [27]:
def get_inverse_perm(perm : list) -> list:
    inv_perm = [0] * len(perm)
    
    for i in range(len(perm)):
        inv_perm[perm[i]] = i
        
    return inv_perm

In [28]:
# Load puzzle_info into a dictionary to easily access the moves

allowed_moves = {}

puzzle_info = pd.read_csv(DATA_DIRECTORY + 'puzzle_info.csv')
for _, row in puzzle_info.iterrows():
    allowed_moves[row['puzzle_type']] = json.loads(row['allowed_moves'].replace("'", '"'))

    # Add inverse moves
    d = allowed_moves[row['puzzle_type']]
    for k, move in tuple(d.items()):
        d['-' + k] = get_inverse_perm(move)

In [29]:
def convert_to_kociemba(s: str) -> str:
    """
    Convert the string to kociemba notation
    1. Change the colors
    2. The faces are processed in the order: 'URFDLB'
    
    Kaggle Coloring:
    (https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023?scriptVersionId=155960527&cellId=14)
             +--------+                               +--------+
             | 0    1 |                               | A    A |
             |   d1   |                               |   d1   |
             | 2    3 |                               | A    A |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
    | 16  17 | 4    5 | 8   9  | 12  13 |    | E    E | B    B | C    C | D    D |
    |   r1   |   f0   |   r0   |   f1   |    |   r1   |   f0   |   r0   |   f1   |
    | 18  19 | 6    7 | 10  11 | 14  15 |    | E    E | B    B | C    C | D    D |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
             | 20  21 |                               | F    F |
             |   d0   |                               |   d0   |
             | 22  23 |                               | F    F |
             +--------+                               +--------+
    
    Kociemba Coloring:
    (https://github.com/muodov/kociemba?tab=readme-ov-file#cube-string-notation)
                 |************|
                 |*U1**U2**U3*|
                 |************|
                 |*U4**U5**U6*|
                 |************|
                 |*U7**U8**U9*|
                 |************|
     ************|************|************|************
     *L1**L2**L3*|*F1**F2**F3*|*R1**R2**R3*|*B1**B2**B3*
     ************|************|************|************
     *L4**L5**L6*|*F4**F5**F6*|*R4**R5**R6*|*B4**B5**B6*
     ************|************|************|************
     *L7**L8**L9*|*F7**F8**F9*|*R7**R8**R9*|*B7**B8**B9*
     ************|************|************|************
                 |************|
                 |*D1**D2**D3*|
                 |************|
                 |*D4**D5**D6*|
                 |************|
                 |*D7**D8**D9*|
                 |************|
    """
    
    convert_color = {'A' : 'U', 
                     'B' : 'F', 
                     'C' : 'R',
                     'D' : 'B',
                     'E' : 'L',
                     'F' : 'D'}
    
    s = list(s)
    m = len(s) // 6
    
    # 1
    for i in range(len(s)):
        s[i] = convert_color[s[i]]
    
    # 2
    s = [s[m*i:m*(i+1)] for i in range(6)]
    perm = [0, 2, 1, 5, 4, 3]
    s = [s[perm[i]] for i in range(6)]
    s = [s[i//m][i%m] for i in range(6*m)]
    
    return ''.join(s)

assert(convert_to_kociemba('AAAABBBBCCCCDDDDEEEEFFFF') == 'UUUURRRRFFFFDDDDLLLLBBBB')

In [30]:
def get_cube(kociemba_notation: str) -> RubiksCube:
    """
    Gets the correct cube object from kociemba notation
    """
    
    size = int((len(kociemba_notation) // 6) ** 0.5) # No precision errors since float result rounded from exact result
    KOCIEMBA_FACE_ORDER = 'URFDLB'
    
    if size == 2:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube222 import RubiksCube222

        cube = RubiksCube222(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 3:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube333 import RubiksCube333

        cube = RubiksCube333(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 4:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube444 import RubiksCube444

        cube = RubiksCube444(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 5:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube555 import RubiksCube555

        cube = RubiksCube555(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 6:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube666 import RubiksCube666

        cube = RubiksCube666(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 7:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube777 import RubiksCube777

        cube = RubiksCube777(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size % 2 == 0:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCubeNNNEven import RubiksCubeNNNEven

        cube = RubiksCubeNNNEven(kociemba_notation, KOCIEMBA_FACE_ORDER)
    else:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCubeNNNOdd import RubiksCubeNNNOdd

        cube = RubiksCubeNNNOdd(kociemba_notation, KOCIEMBA_FACE_ORDER)
    
    return cube

In [31]:
def convert_cuber_moves_to_kaggle_moves(moves : list[str], n) -> list[str]:
    """
    Converts the cuber move to kaggle moves
    
    To learn about cuber notation for moves:
        https://www.youtube.com/watch?v=37y4f8FYdFs
        One confusing case is that Rw = 2Rw (they are used interchangeably)
    
    To learn about kaggle moves:
        https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023?scriptVersionId=155960527&cellId=14
        Note that wide moves don't exist on kaggle
    
    Major discrepancy:
        There is a kaggle move that rotates each layer clockwise with respect to 3 the
        FRD faces.
        However, cuber moves are clockwise with respect to one of the 6 faces.
        
    Examples:
        The R move corresponds to r_0
        The L corresponds to -r_{n-1}.
        The R2 move corresponds to r_0, r_0
        The Rw or 2Rw move corresponds to r_0, r_1
        The 2Lw move corresponds to -r_{n-1}, -r_{n-2}
    """
    
    convert_axis = {'R' : 'r',
                    'L' : 'r',
                    'F' : 'f',
                    'B' : 'f',
                    'D' : 'd',
                    'U' : 'd'}
    axis_changed = 'LBU'
    invert_str = ['', '-']
    
    
    kaggle_moves = []
    
    for move in moves:
        rotations = 1
        invert = 0
        
        if move[-1] == "'":
            invert = 1
        if move[-1] == '2':
            rotations = 2
        
        width = 1 # If no w, just rotate single layer
        
        if 'w' in move:
            width = 2  # Default if no number
            if move[0].isdigit():
                # Get and remove width from prefix of move
                width = ''
                for i in range(len(move)):
                    if not move[i].isdigit():
                        width = int(move[:i])
                        move = move[i:]
                        break

        original_axis = move[0]
        axis = convert_axis[original_axis]
        
        # Get kaggle moves
        for i in range(width):
            for j in range(rotations):
                if original_axis in axis_changed:
                    kaggle_moves.append(invert_str[invert^1] + axis + str(n-1-i))
                else:
                    kaggle_moves.append(invert_str[invert] + axis + str(i))
    
    return kaggle_moves

In [32]:
def add_final_rotation(puzzle_type: str,
                       initial_state: str,
                       moves: list[str]) -> None:
    """
    The solver from GitHub doesn't care about the final orientation of the cube.
    We'll need to rotate it.
    
    I think this uses the minimum rotations (should double-check).
    Should take at most 3*n kaggle moves (3 rotations).
    
    Updates moves in place.
    """

    n = int((len(initial_state) // 6) ** 0.5)
    m = n*n

    current_array = np.array(tuple(initial_state))
    
    for move in moves:
        current_array = current_array[allowed_moves[puzzle_type][move]]

    new_moves = []
    b_at_3 = False
    
    # Get B in place
    # Rotation[i] = Move(s) to get B in place if they are at face i in kaggle order
    rotation = [('-r',), tuple(), ('-d',), ('d', 'r'), ('d',), ('r',)]  
    for i in range(6):
        if current_array[i*m] == 'B' and rotation[i]:
            b_at_3 = i == 3
            rotations = 2 if i == 3 else 1  # B is opposite where it should be
            for j in range(rotations):
                new_moves.extend(rotation[i][0] + str(k) for k in range(n))
    
    # Apply moves
    current_array = current_array.copy()
    for move in new_moves:
        current_array = current_array[allowed_moves[puzzle_type][move]]
    
    # Get 'C' in place
    if b_at_3 and current_array[4*m] == 'C':
        # We rotated on i = 3 to get B in place.
        # If we use the other valid rotation, it puts B AND C in place.
        new_moves = []
        for j in range(2):
            new_moves.extend(rotation[3][1] + str(k) for k in range(n))
    else:
        rotation = ['f', None, '', None, 'f', '-f']  # Notice how we only rotate about f axis
        for i in range(6):
            if current_array[i*m] == 'C' and rotation[i] != '':
                rotations = 2 if i == 4 else 1  # C is opposite where it should be 
                for j in range(rotations):
                    new_moves.extend(rotation[i] + str(k) for k in range(n))
    
    moves += new_moves

In [33]:
def invert_kaggle_moves(moves : list[str]) -> list[str]:
    """
    Gives a list of moves that when applied in order has the inverse effect
    of the original list of moves applied in order
    """
    
    inverse_moves = []
    
    for move in moves[::-1]:
        if move[0] == '-':
            inverse_moves.append(move[1:])
        else:
            inverse_moves.append('-' + move)
        
    return inverse_moves

In [34]:
def get_kaggle_moves(puzzle_type: str,
                     initial_state: str,
                     solution_state: str) -> list[str]:
    """
    Let A be the moves that get us from initial_state to solved Rubik's Cube
    Let B be the moves that get us from solution_state to solved Rubik's Cube
    
    Then to get from initial_state to solution_state, apply A then inverse(B)
    """
    
    initial_state_kociemba = convert_to_kociemba(initial_state)
    solution_state_kociemba = convert_to_kociemba(solution_state)

    # I edited RubiksCube222, so it would return instead of sys.exit(0) (error in jupyter)
    # Also removed the unnecessary print right above that line
    initial_cube = get_cube(initial_state_kociemba)
    solution_cube = get_cube(solution_state_kociemba)

    initial_cube.solve()
    solution_cube.solve()
    
    initial_cuber_moves = initial_cube.get_solution()  # Custom method I added
    solution_cuber_moves = solution_cube.get_solution()  # Custom method I added

    n = int((len(initial_state) // 6) ** 0.5)
    initial_kaggle_moves = convert_cuber_moves_to_kaggle_moves(initial_cuber_moves, n)
    solution_kaggle_moves = convert_cuber_moves_to_kaggle_moves(solution_cuber_moves, n)
    
    add_final_rotation(puzzle_type, initial_state, initial_kaggle_moves)
    add_final_rotation(puzzle_type, solution_state, solution_kaggle_moves)


    return initial_kaggle_moves + invert_kaggle_moves(solution_kaggle_moves)

In [35]:
def check_moves(puzzle_type: str,
                initial_state : str,
                solution_state : str,
                moves : list[str]) -> bool:
    """
    Returns if moves converts initial_state to solution_state
    """
    
    current_array = np.array(tuple(initial_state))
    solution_array = np.array(tuple(solution_state))
    
    for move in moves:
        current_array = current_array[allowed_moves[puzzle_type][move]]
    
    return (current_array == solution_array).all()

In [36]:
# Get submission and cube puzzles

submission = pd.read_csv(DATA_DIRECTORY + 'submission.csv', index_col=0)
puzzles = pd.read_csv(DATA_DIRECTORY + 'puzzles.csv')
puzzles = puzzles[puzzles['puzzle_type'].str.contains('cube')]

In [37]:
# To quantify improvement and potential improvement

skipped_has_n = []  # Skipped because has 'N'
tot_skipped_has_n = 0
cannot_solve = []  # Skipped because meeting point of standard solved cube impossible
tot_cannot_solve = 0
improve = {}
tot_improved = 0
error_while_solving = []  # Unknown errors, but almost certainly from GitHub cube solver
cube_solver_wrong = []  # Cube solver doesn't get it right...

In [38]:
# 130..
# 200, 201, 202, 203, 204 (4x4x4)
# 235, 236, 237, 238, 239, (5x5x5) error while solving,
# 200-204 are all the even cubes with ABA solution.
# (Could not determine wing_str for (F, B))
redos = [282]
redo = [235]
# error_while_solving=[(282, 'cube_33/33/33', 139625, Exception('Could not determine wing_str for (B, F)'))]

for i in tqdm(list(range(len(puzzles)))):
    initial_state = puzzles.iloc[i]['initial_state'].split(';')
    solution_state = puzzles.iloc[i]['solution_state'].split(';')
    puzzle_type = puzzles.iloc[i]['puzzle_type']
    id = puzzles.iloc[i]['id']
    old_moves = submission.iloc[id]['moves'].split('.')

    if 'N' in initial_state[0]:
        continue
        # Has 'N'
        # skipped_has_n.append((id, puzzle_type, len(old_moves)))
        # tot_skipped_has_n += len(old_moves)
        order = 'ABCDEF'
        m = len(initial_state) // 6
        
        initial_state = str(order[int(x[1:])//m] for x in initial_state)
        solution_state = str(order[int(x[1:])//m] for x in solution_state)
    else:
        initial_state = ''.join(initial_state)
        solution_state = ''.join(solution_state)
    
    # try:
    moves = get_kaggle_moves(puzzle_type, initial_state, solution_state)
        ## Impossible to get to meeting point of standard solved cube
        #cannot_solve.append((id, puzzle_type, len(old_moves)))
        #tot_cannot_solve += len(old_moves)
        #continue
    # except Exception as e:
    #     # Almost certainly an issue with the GitHub cube solver
    #     error_while_solving.append((id, puzzle_type, len(old_moves), e))
    #     continue
        
    # Verify correctness
    if not check_moves(puzzle_type, initial_state, solution_state, moves):
        cube_solver_wrong.append((id, puzzle_type, len(old_moves)))
    
    # Compare and replace if better
    if len(moves) < len(old_moves):
        submission.loc[id, 'moves'] = '.'.join(moves)  # Replace
        
        improvement = len(old_moves) - len(moves)
        # Print improvement
        print(f'Improved {puzzle_type} with id {id} by {improvement} moves')
        
        # Store improvement statistics
        if puzzle_type not in improve:
            improve[puzzle_type] = [0, 0, 0]
        improve[puzzle_type][0] += improvement
        improve[puzzle_type][1] += len(old_moves)
        improve[puzzle_type][2] += 1
        
        tot_improved += improvement

  0%|          | 0/284 [00:00<?, ?it/s]

UUUURRRRFFFFDDDDLLLLBBBB
UUUURRRRFFFFDDDDLLLLBBBB
UUUURRRRFFFFDDDDLLLLBBBB
UUUURRRRFFFFDDDDLLLLBBBB
UUUURRRRFFFFDDDDLLLLBBBB
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 140 by 174 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 141 by 240 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 142 by 338 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 143 by 396 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 144 by 304 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 145 by 224 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 146 by 296 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 with id 147 by 210 moves
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Improved cube_3/3/3 

found 50 phase4 solutions with --solution-count 50, 2 solutions without PLL
phase 3 is 7 steps, phase 4 is 9 steps, solve 333 in 21 steps, total 37 (NEW MIN)
phase 3 is 7 steps, phase 4 is 9 steps, solve 333 in 22 steps, total 38


Improved cube_4/4/4 with id 205 by 458 moves
UUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBB


found 50 phase4 solutions with --solution-count 50, 17 solutions without PLL
phase 3 is 8 steps, phase 4 is 10 steps, solve 333 in 22 steps, total 40 (NEW MIN)
phase 3 is 8 steps, phase 4 is 10 steps, solve 333 in 21 steps, total 39 (NEW MIN)
phase 3 is 8 steps, phase 4 is 10 steps, solve 333 in 22 steps, total 40
phase 3 is 8 steps, phase 4 is 10 steps, solve 333 in 22 steps, total 40
phase 3 is 8 steps, phase 4 is 10 steps, solve 333 in 22 steps, total 40


...

Improved cube_4/4/4 with id 208 by 410 moves
UUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBB


found 50 phase4 solutions with --solution-count 50, 1 solutions without PLL
phase 3 is 6 steps, phase 4 is 11 steps, solve 333 in 21 steps, total 38 (NEW MIN)


Improved cube_4/4/4 with id 209 by 340 moves
UUUUUUUUUUUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBBBBBBBBBBB
Improved cube_5/5/5 with id 240 by 1446 moves
UUUUUUUUUUUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBBBBBBBBBBB
Improved cube_5/5/5 with id 241 by 1222 moves
UUUUUUUUUUUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBBBBBBBBBBB
Improved cube_5/5/5 with id 242 by 1312 moves
UUUUUUUUUUUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBBBBBBBBBBB
Improved cube_5/5/5 with id 243 by 1460 moves
UUUUUUUUUUUUUUUUUUUUUUUUURRRRRRRRRRRRRRRRRRRRRRRRRFFFFFFFFFFFFFFFFFFFFFFFFFDDDDDDDDDDDDDDDDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLBBBBBBBBBBBBBBBBBBBBBBBBB
Improved cube_5/

found 50 phase4 solutions with --solution-count 50, 0 solutions without PLL
found 1050 phase4 solutions with --solution-count 1050, 4 solutions without PLL


Improved cube_6/6/6 with id 256 by 2376 moves
UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU

In [46]:
print(f'{tot_improved=}')

tot_improved=38900


In [40]:
print(f'{tot_skipped_has_n=}')
print(f'{skipped_has_n=}')

tot_skipped_has_n=0
skipped_has_n=[]


In [41]:
print(f'{tot_cannot_solve}')
print(f'{cannot_solve=}')

0
cannot_solve=[]


In [42]:
print(f'{error_while_solving=}')
print(f'{cube_solver_wrong=}')

error_while_solving=[]
cube_solver_wrong=[]


In [43]:
redo = [a[0] for a in error_while_solving] + [a[0] for a in cube_solver_wrong]
print(f'{redo=}')

redo=[]


In [44]:
for puzzle_type, v in improve.items():
    print(f'{puzzle_type} Average Improvement: {v[0]/v[2]} Score, {100*v[0]/v[1]}%')

cube_3/3/3 Average Improvement: 255.8 Score, 87.72290809327846%
cube_4/4/4 Average Improvement: 391.6 Score, 79.91836734693878%
cube_5/5/5 Average Improvement: 1386.8 Score, 88.34246400815391%
cube_6/6/6 Average Improvement: 2376.0 Score, 87.09677419354838%
cube_33/33/33 Average Improvement: 25074.0 Score, 22.99523110785033%


In [45]:
# Save submission

submission.to_csv(DATA_DIRECTORY + 'submission.csv')